In [1]:
%%HTML
<link rel="stylesheet" type="text/css" href="https://fonts.googleapis.com/css?family=Quicksand:300,700" />
<link rel="stylesheet" type="text/css" href="https://fonts.googleapis.com/css?family=Fira Code" />
<link rel="stylesheet" type="text/css" href="rise.css">

In [ ]:
import ipytest
ipytest.config(rewrite_asserts=True, magics=True)

DATA_DIR = "../data"

__file__ = "9-testing.ipynb"

# Testing in PySpark

![footer_logo_new](images/logo_new.png)

## Testing

Testing is incredibly important when writing code and helps you verify that your code does what you expect. 

We won't focus on why you should test but rather how to write *unit tests*.

Keep in mind that sometimes having good tests can reduce the amount of documentation you need to write.

### Inline tests

Tests can be written inline with your code like this:

```python
def coalesce_nan(a, b):
    if (a and not np.isnan(a)):
        return a
    return b

def test_coalesce():
    assert coalesce_nan(None, 3) == 3
    assert coalesce_nan(2, 3) == 2
    assert coalesce_nan(np.nan, 3) == 3
    assert coalesce_nan(np.nan, "my") == "my"
```

This has several advantages: no imports to manage, and if you use a tool such as `py.test`, it can recognize that it should only call the function with the `test_` prefix.

### Tests in a separate directory

Alternatively, when writing a Python package, you could have the following directory structure (for example)

```sh
setup.py
conftest.py
my_module/
    __init__.py
    app.py
    helper.py
tests/
    helper_tests.py
docs/
    ...
```

Using `pytest` (`pip install pytest`) you could run the tests by typing `pytest` from the command line. The framework expects tests to be located in `tests/`, and assumes that the methods to be tested should begin with `test_`

## Testing in PySpark

To test your PySpark code, you need to put in some extra effort to setup a Spark instance for your tests.

### Setting up Spark 

In `pytest`, we can setup a Spark instance by defining a fixture:

In [ ]:
import pyspark
import pytest

@pytest.fixture(scope="session")
def spark():
    """Returns a Spark instance for testing."""
    return ( 
        pyspark.sql.SparkSession.builder
        .master("local[2]") 
        .getOrCreate()
    )

Once defined, fixtures can be passed to tests by using an input argument with the same name as the fixture:

In [ ]:
%%run_pytest[clean]

import pandas as pd


def test_example(spark):
    """Example test using the Spark fixture."""
         
    data = pd.DataFrame({
        "a": [1, 2, 3],
        "b": ["a", "b", None]
    })
    example_df = spark.createDataFrame(data)
    
    assert example_df.count() == 3

For more information on pytest fixtures, see: https://docs.pytest.org/en/latest/fixture.html.

### Test datasets

Fixtures can also be used to define reusable test datasets:

In [ ]:
%%run_pytest[clean]

@pytest.fixture()
def example_df(spark):
    """Example dataframe."""
    
    data = pd.DataFrame({
        "a": [1, 2, 3],
        "b": ["a", "b", None]
    })
    
    return spark.createDataFrame(data)


def test_example_fixture(example_df):
    """Example test using a test dataset from a fixture."""
    assert example_df.count() == 3

Of course you can also read a test dataset from a static input file:

In [ ]:
%%run_pytest[clean]

import os

@pytest.fixture(scope="session")
def airline_df(spark):
    """Rather large static dataset containing airline data."""
    data_path = os.path.join(DATA_DIR, 'airlines.parquet')
    return spark.read.parquet(data_path)


def test_example_fixture_heroes(airline_df):
    assert airline_df.count() > 1000

## How/what to test?

In general, you should aim to write tests that (at the very least) cover the parts of your code that define your computations.

You can make your code relatively easy to test by decomposing it into concise functions that take an input dataframe and return an output dataframe.

This allows you to pass an example (test) dataframe to the function and verify the result(s). 

Try to structure your test dataframe(s) to test different edge cases in your function (such as null handling, etc.).

## Exercise - Testing

1. Load the heroes dataset and write a function that filters the dataset for a given role:

```python
def filter_for_role(heroes_df, role):
    ...
```

2. Write a unit test that applies your function for the 'Warrior' role and checks if the number of returned rows is what you would expect. Try to use a fixture for passing the test dataset to your unit test.

3. Write a function that counts the number of occurrences of each role in a given dataset:

``` python
def count_roles(heroes_df):
    ...
```
4. Write a unit test that checks the result of the `count_roles` function on our test dataset.

In [ ]:
%load ../answers/02_testing.py